**Single Cell Resolution Analysis of Ca Imaging Data for Hydra v4** 
=============================================================
*(Use with data from tdTomato_GCamP Animals)*

Requires:
-------------------------
- Green and Red Channel Videos From 2-Colour Confocal (GCaMP Channel and tdTomato)
- Tracking Position Information From ICY Spot Tracking Protocol (Exported as CSV)
- Conda Environment: Caiman_NOAH3
- SCHyA .py file

INPUT PATHS
-----------------
Enter paths to the appropriate files                    
**Videos must be .avi (convert in imageJ if not)**     
**Can also use a Tif sequence folder - change the read data function to Read_Data_TIFseq for this**    
**If using windows**: Paths must start with an 'r' character: e.g. vid_path = **r**"C:\Users\rylab\ path to your file \clip.avi"

In [1]:
#input path to your .csv from ICY
csv_path = '/Users/noah/Documents/Columbia_Project/thesis_dump/VID1_oct_4_3hz/AVIs_and_CSV/points.csv'

#input path to .avi of GCaMP Video
vid_path = '/Users/noah/Documents/Columbia_Project/thesis_dump/VID1_oct_4_3hz/AVIs_and_CSV/green(1-852).avi'

#input path to .avi of tdTomato Video
red_vid_path = '/Users/noah/Documents/Columbia_Project/thesis_dump/VID1_oct_4_3hz/AVIs_and_CSV/red(1-852).avi'


SET UP DATA  
-----------------------
Run the following cells to set up the 
data to be analysed        
**Set the FFmpeg path before running this cell**

In [2]:
#importing packages

import SCHyA as hy

/Users/noah/anaconda3/envs/syfi/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.decomposition.incremental_pca module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/noah/anaconda3/envs/syfi/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/noah/anaconda3/envs/syfi/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood a

In [3]:
#reading in data
positions, vid, red_vid = hy.Read_Data(csv_path, vid_path, red_vid_path)

In [4]:
#reshape the .csv into a more useable form
posit = hy.reshaper(positions)

In [5]:
#Get Information About Raw Data
num_frames = len(vid)
num_red_frames = len(red_vid)
num_tracks = int(positions[len(positions)-1,0])
#Display Information
print('Numer of Frames in Green Video:', num_frames)
print('Number of Frames in Red Video:', num_red_frames)
print('Number of Points Tracked by ICY:', num_tracks)

Numer of Frames in Green Video: 852
Number of Frames in Red Video: 852
Number of Points Tracked by ICY: 713


CALCIUM SIGNAL EXTRACTION
------------------------
These following cells will remove neurons that were not completely tracked throughout the video, extract a region of interest around each fully tracked neuron, and extract the intensity of each neuron throughout the video and plot the raw intensities. The signal can then be corrected for motion artefacts by finding the ratiometric signal between the green and red channels - dR/R; or through the use of Independant Component Analysis. 

In [6]:
#Discard Neurons which were incompletely tracked
posit_corrected = hy.remove_incomplete_tracks(posit, num_frames)

The Following 2 Cells may Give a Runtime Warning - This is Fine

**Must Run the Extract_Fluorescence function before the SingleCellIntensities function as you need to use the updated 'posit_corrected' output from Extract_Fluorescence as SingleCellIntensities as no feature to correct this itself**

In [7]:
#Extract the fluorescence intensity in the GCaMP channel throughout the video for each neuron

#size of ROI
dim = 9
subROI_Circle_size = 4
LookBack = 4 #resistance of tracking to movement i.e. how quickly a tracked neuron can move before position resets

intensities1, posit_corrected1 = hy.Extract_Fluorescence(posit_corrected, vid, dimention = dim)

intensities1, posit_corrected1, neuron_pts = hy.SingleCellIntensities(vid, posit_corrected1, dim, subROI_Circle_size, LookBack)

/Users/noah/anaconda3/envs/syfi/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/noah/anaconda3/envs/syfi/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


NameError: name 'posit' is not defined

In [ ]:
#Extract the fluorescence intensity in the Red channel throughout the video for each neuron

intensities_red, posit_corrected = Extract_Fluorescence(posit_corrected1, red_vid, dim)

# intensities_red = SingleCellIntensities_Red(red_vid, neuron_pts, posit_corrected, dim, 4)

In [ ]:
#plot raw neuronal intensities & display number of fully tracked neurons
number_of_neurons = len(intensities1)
print('number of neurons = ', number_of_neurons)

for track in range(len(intensities1)):
    plt.plot(intensities1[track])
    plt.title('Raw GCaMP Intensity plots of all sufficiently recorded neurons')
    plt.xlabel('Frame')
    plt.ylabel('Intenstiy')
plt.show()

#Plot intensities of red channels - used to identify motion artefacts and leaking between channels
for track in range(len(intensities1)):
    plt.plot(intensities_red[track])
    plt.title('Intensity Plot of Red Channel')
    plt.xlabel('Frame')
    plt.ylabel('Neuron')

In [ ]:
#DeltaR/R or ICA used to correct for motion artefacts and cross channel leaking
#Obtain Artefact Free Ca Signal
CaSignal = ICAdecorr(intensities1, intensities_red, 0.5, 10)

#Plot Signal
for i in range(len(CaSignal)):
    plt.plot(CaSignal[i])
plt.title('Extracted Calcium Signal Plot')
plt.xlabel('Frame')
plt.ylabel('Ca Intensity')
plt.show()

In [ ]:
#Plot Heatmap of Calcium Signal
#can normalise here for better signal...
plot_heatmap(CaSignal, 'Heatmap of Extracted Calcium Signal', 'Ca Signal Intensity')

Calcium Signal Extraction - Optional Cells
--------------------------
**These Cells Dont Need to be Run**          
Cells for evaluating performance or saving preliminary results        
Only Run the Cells you want to

**Optional Cells to Evaluate ROI Tracking**      
Input frame number to view all of the ROIs tracked in that frame      

In [ ]:
#superimpose ROIs on video frame in both channels

#select frame to view
frame_to_view = 510

Super_impose(vid, frame_to_view, 'Neuron Tracking in Green Channel')
Super_impose(red_vid, frame_to_view, 'Neuron Tracking in Red Channel')

**Optional Cells to Evaluate Effect and Correction of Motion Artifacts and Red Leakage**

In [ ]:
#Plot Raw Transients in Green
plot_all(intensities)

In [ ]:
#Plot Red Transient to check for motion artefacts and leaking
Neuron_with_possible_artefact = 52

plt.figure(1)
plt.title('Raw Red Channel Plot')
plt.plot(intensities_red[Neuron_with_possible_artefact])
plt.show()

#After Correction:
plt.figure(2)
plt.plot(CaSignal[Neuron_with_possible_artefact])
plt.title('Calcium Signal Plot')
plt.show()

**Optional Cell to Evaluate Activity and Tracking of Individual Neurons**     
Allows comparison between intensity trace of a neuron and that neuron's ROI at a particular frame    
Input Neuron Number and Frame Number

In [ ]:
#input neuron to check
neuron = 42

#input frame form video to view the ROI
eval_frame = 509

full_eval(neuron, CaSignal, eval_frame, dim, posit_corrected)

FILTERING & SMOOTHING
-----------------------

The following cells will filter the intensity traces of the neurons to remove signal that are likely from nematocytes, smooth the results to reduce the appearance of noise, and detrend the resulting data

In [ ]:
#Detrending data - use on filtered or smoothed data

#Set Polynomial Degree
poly_deg = 17

detrended = detrend_all(CaSignal, poly_deg)

In [ ]:
#Filter Nematocytes

#set the threshold to remove below
#standard option is to set to 20 (i.e. filter will remove signals with below the 20th percentile of standard deviations)

# percentile_threshold = 0 #zero = no filtering
# filt, posit_corrected = filt_nematocytes(CaSignal,percentile_threshold,posit_corrected)

alpha = 1e-6 #Tune threshold for coherence to gaussian distribution - need to use detrended data with this function
detrended, posit_corrected, removed = Gaussian_noise_filter(detrended, alpha, posit_corrected)

In [ ]:
#evaluate filtering of neurons

plot_all(removed)

In [ ]:
#Smooth Signals

# #Set Smoothing window Size
# window = 2

# smooth_intensities = smoother(filt, window)
# detrended = smooth_intensities

In [ ]:
#Display Filtering and Smoothing Results

#Display Total Number of Neurons After Filtering
print('no of neurons: ', len(detrended))

# plt.figure(1)
# for i in range(len(filt)):
#     plt.plot(filt[i])
# plt.title('Calcium Signal with Nematocytes Filtered')
# plt.xlabel('Frame')
# plt.ylabel('Calcium Signal')
# plt.show()

# plt.figure(2)
# for i in range(len(smooth_intensities)):
#     plt.plot(smooth_intensities[i])
# plt.title('Smoothed Filtered Fluorescence with Single Neuron Resolution')
# plt.xlabel('Frame')
# plt.ylabel('Calcium Signal')
# plt.show()

plt.figure(3)
for i in range(len(detrended)):
    plt.plot(detrended[i])
plt.title('Detrended Calcium Signal Data')
plt.xlabel('Frame')
plt.ylabel('Calcium Signal Detrended')
plt.show()

plot_heatmap(norm_all_data(detrended), 'Heatmap of Filtered Detrended Data', 'Calcium Signal Intensity')

In [ ]:
#Plot all remaining neurons after filtering and processing
plot_all(detrended)

CALCIUM SIGNAL ANALYSIS
-------------------------------------

The following cells analyse the results to show extract spikes from the Calcium intensity data

**Tune Threshold used to Indentify Spikes to Fit Data Using Evaluation Cell for FOOPSI at End of Script**

In [ ]:
#Create Raster Plot using CAIMAN's FOOPSI function - Denoising and Deconvolution

#foopsi
Foopsi_ca, spikes_signal_dR = FOOPSI_all(detrended)

#Extract Raster Plot Data

#Threshold (could use a theoretical value for threshold! - See CAIMAN Docs - but trial & error is also fine)
#USE FOOPSI EVALUATION CELL TO TUNE THIS PARAMETER (0.04 works well)
spike_thresh_dR = 0.2

raster_array_dR = Find_Raster_adaptive(spikes_signal_dR, spike_thresh_dR)

In [ ]:
#Plot all denoised calcium traces
plot_all(Foopsi_ca)

In [ ]:
plot_heatmap(norm_all_data(Foopsi_ca), 'Denoised heatmap', 'Intensity')

In [ ]:
#Display Results

#plot raster
#zero values from way array was made array give large syncronous train at start of the signal - should fix (.append method?)
plt.figure(2)
plt.eventplot(raster_array_dR,linelengths = 0.6)
plt.ylabel('Neuron')
plt.xlabel('Frame')
plt.title('Raster Plot of Neural Activity of Hydra')
plt.xlim((1,len(raster_array_dR[0])))
plt.show()

In [ ]:
#plot rasterplot for all neurons
for i in range(len(raster_array_dR)):
    plt.eventplot(raster_array_dR[i],linelengths = 0.6)
    plt.xlim((1,len(raster_array_dR[0])))
    plt.title(i)
    plt.show()

BEHAVIOUR ANALYSIS WITH NEURAL ACTIVITY
---------------------------------------------
These cells plot neural activity and behaviour together to allow comparision between neural activity and behaviour     
Most of this step still needs to be done by hand unfortunately...

In [ ]:
#Input Behaviour Arrays

#behaviour array: [frame behaviour turns on,frame off,on,off,on,... etc]
behaviour_frames = [0,212,605,len(detrended[0])]

#fill with numbered behaviourws in order they occur
behaviours = [1,0,1]

#fill with colours corresponding to numbered behaviours
colours=['blue','green']

In [ ]:
#Display Neural Activity with Behaviour

plt.figure(1)
fig, ax = plt.subplots()
ax.margins(0.02)
for behav in range(len(behaviour_frames)-1):
    colour_face = colours[behaviours[behav]]
    ax.axvspan(behaviour_frames[behav],behaviour_frames[behav+1], facecolor = colour_face, alpha = 0.2)
for i in range(len(intensities)):
    plt.plot(intensities[i])
plt.title('Raw Intensities plotted with CB and RP behaviour (CB=green)')
plt.xlabel('Frame')
plt.ylabel('Intensity')
plt.show()

plt.figure(2)
fig, ax = plt.subplots()
ax.margins(0.02)
for behav in range(len(behaviour_frames)-1):
    colour_face = colours[behaviours[behav]]
    ax.axvspan(behaviour_frames[behav],behaviour_frames[behav+1], facecolor = colour_face, alpha = 0.2)
for i in range(len(detrended)):
    plt.plot(detrended[i])
plt.title('Processed Intensities plotted with CB and RP behaviour (CB=green)')
plt.xlabel('Frame')
plt.ylabel('Calcium Signal Intensities Detrended')
plt.show()

plt.figure(3)
fig, ax = plt.subplots()
ax.margins(0.02)
for behav in range(len(behaviour_frames)-1):
    colour_face = colours[behaviours[behav]]
    ax.axvspan(behaviour_frames[behav],behaviour_frames[behav+1], facecolor = colour_face,alpha = 0.2)
plt.eventplot(raster_array_dR,linelengths = 0.6)
plt.xlim((1,len(raster_array_dR[0])))
plt.ylabel('Neuron')
plt.xlabel('Frame')
plt.title('Raster Plot of Neural Activity of Hydra Plotted with Behaviour')
plt.show()

CORRELATION AND INTERPRETATION OF DATA
-----------------------------------
The following cells work to group neurons together into ensembles, display various froms of correlation data presentation, and correlate each neuron to different types of behaviour

**Input Correlation Threshold to be Used to Group Neurons**

In [ ]:
#Create Dendogram (can change method between ward, average, etc)
CaSignal_norm = spikes_signal_dR
plt.figure(figsize=(10, 7))  
plt.title("Dendrograms")  
dend = shc.dendrogram(shc.linkage(CaSignal_norm, method='ward'))

**Input Number of Clusters Based on Results from Dendogram**

In [ ]:
#extract and view neurons from each cluster 

#number of clusters chosen from examination of dendogram
number_of_clusters = 5

clusters = give_cluster_assigns(pd.DataFrame(CaSignal_norm), number_of_clusters, transpose=False)
clusters = clusters.values

#plot specific cluster's neurons traces

cluster_to_view = 2

for i in range(len(clusters)):
    if clusters[i] == cluster_to_view:
        plt.figure(i-1)
        plt.title(i)
        plt.plot(Foopsi_ca[i])
        plt.show()

In [ ]:
#Super impose locations of neurons in specified Cluster onto video frame

cluster_to_view = 1
frame_for_cluster = 30
video_for_cluster = vid
title_for_cluster = 'Cluster Positions'

Super_impose_cluster(video = video_for_cluster, frame_to_view = frame_for_cluster, posit_corrected = posit_corrected, clusters = clusters, cluster_to_view = cluster_to_view, Title = title_for_cluster)

In [ ]:
#ClusterMap
#change figure size to see all labels if necessary

df = pd.DataFrame(np.transpose(CaSignal_norm))

# Draw the full plot
sns.clustermap(df.corr(), center=0, cmap="coolwarm",linewidths=.75, figsize=(19, 19), method = 'ward')

In [ ]:
#Investigate Correlations from ClusterMap

#input neurons to view
neuron_1 = 7
neuron_2 = 14

plt.figure(1)
fig, ax = plt.subplots()
ax.margins(0.02)
for behav in range(len(behaviour_frames)-1):
    colour_face = colours[behaviours[behav]]
    ax.axvspan(behaviour_frames[behav],behaviour_frames[behav+1], facecolor = colour_face,alpha = 0.2)
plt.plot(detrended[neuron_1])
plt.plot(detrended[neuron_2])
plt.show()

plt.figure(2)
plt.title(neuron_1)
plt.eventplot(raster_array_dR[neuron_1],linelengths = 0.6)
plt.xlim((1,len(raster_array_dR[0])))
plt.show()

plt.figure(3)
plt.title(neuron_2)
plt.eventplot(raster_array_dR[neuron_2],linelengths = 0.6)
plt.xlim((1,len(raster_array_dR[0])))
plt.show()

In [ ]:
#Single neuron investigation

neuron_to_investigate = 33
frame_to_investigate = 200

single_neuron_investigation(neuron_to_investigate, CaSignal, vid, frame_to_investigate, dim, posit_corrected)

**Save Data**
===============
**Optional Cells to Save Results**        
Run these cells to save the data as a .csv         
Input file name and path where file should be saved           
**On Windows: Paths still need to start with 'r'**              
e.g. r'C:\Users\rylab\Documents\path to your folder\title.csv'

**Save Raw GCamp Intensity Data**

In [ ]:
#Save Raw GCamp Intensity data as a .csv 

# Save_Path = r'C:\Users\rylab\Documents\Noah2019_desktop\Columbia_Project\hydra_Vids\2-colour\single_cell_intensity_behaviour_longclip.csv'

# intensity_dataframe = pd.DataFrame(intensities)
# intensity_dataframe.to_csv(Save_Path)

**Save CaSignal Intensity Data**

In [ ]:
#save CaSignal data to .csv

# Ca_save_path = r'C:\Users\rylab\Documents\Noah2019_desktop\Columbia_Project\hydra_Vids\2-colour\single_cell_CaSig.csv'

# intensity_dataframe = pd.DataFrame(CaSignal)
# intensity_dataframe.to_csv(Ca_save_path)

**Sava Detrended Intensity Data**

In [ ]:
#save detrended data to .csv

# dR_detrend_save_path = r'C:\Users\rylab\Documents\Noah2019_desktop\Columbia_Project\hydra_Vids\2-colour\single_cell_detrend.csv'

# dfintensity_dataframe = pd.DataFrame(detrended)
# dfintensity_dataframe.to_csv(dR_detrend_save_path)

**Save Foopsi De-noised Data**

In [ ]:
# #save Foopsi de-noised data to .csv

# Ca_denoised_path = r'C:\Users\rylab\Documents\Noah2019_desktop\Columbia_Project\Project_Behaviour_and_Neural_Activity\Single_Cell_Analysis_Neural\Single_Cell_Data\VID1_oct_4_3hz\Results\denoised_neuraldata.csv'

# ca_dataframe = pd.DataFrame(Foopsi_ca)
# ca_dataframe.to_csv(Ca_denoised_path)

**Save Raster Plot Data**

In [ ]:
#save raster data to .csv (needs to be reformatted from event plot form first)

# raster_save_path = r'C:\Users\rylab\Documents\Noah2019_desktop\Columbia_Project\hydra_Vids\2-colour\single_cell_rasterb1.csv'

# raster_array_pred = np.zeros((len(raster_array_dR),len(raster_array_dR[1])))
# for i in range(len(spikes_signal_dR)):
#     for j in range(len(spikes_signal_dR[i])):
#         if max(spikes_signal_dR[i]) > 0:
#             if spikes_signal_dR[i][j] >= spike_thresh_dR: #*np.mean(spikes_signal_dR[i]):
#                 raster_array_pred[i][j] = 1

# raster_dataframe = pd.DataFrame(raster_array_pred)
# raster_dataframe.to_csv(raster_save_path)

**Save Behaviour Data**

In [ ]:
#Save Behaviour Array tp .csv
# behaviour_Save_Path = r"C:\Users\rylab\Downloads\behav_array.csv"

# behav_array = np.zeros(len(vid))
# for i in range(len(behaviours)):
#     behav_array[behaviour_frames[i]:behaviour_frames[i+1]] = behaviours[i]
    
# behaviour_dataframe = pd.DataFrame(behav_array)
# behaviour_dataframe.to_csv(behaviour_Save_Path)

**Evaluation Cells**
====================
These cells allow for the evaluation of the tuning of various functions

**Evaluate Polynomial Detrending Parameters** 

In [ ]:
#evaluate ploynomail detrending
neuron_test_detrend = 10
#polynomial degree of 15-17 gives good results - but tune for new data!
polynomial_test_degree  = 12
detrend_one = detrend(smooth_intensities[neuron_test_detrend],polynomial_test_degree)
plt.figure(1)
plt.plot(detrend_one, c='b')
plt.plot(smooth_intensities[neuron_test_detrend], c = 'r')
plt.show()

**Evaluste ICA vs Ratiometric Artefact Removal**

In [ ]:
# New ICA vs Ratiometric

a, b = np.asanyarray(norm_Data(intensities[12])), np.asanyarray(norm_Data(intensities_red[12]))
plt.plot(a, c = 'g')
plt.title('Raw Intensities of GCaMP7 (Green) Channel')
plt.xlabel('Frames')
plt.ylabel('Normalised Intensity')
plt.show()
plt.plot(b, c = 'r')
plt.title('Raw Intensities of tdTomato (Red) Channel')
plt.xlabel('Frames')
plt.ylabel('Normalised Intensity')
plt.show()

c = (a+1)/(b+1)
plt.plot(c)
plt.title('Ratiometric Correction of Motion Artefacts')
plt.xlabel('Frames')
plt.ylabel('Intensity Ratio')
plt.show()

print(min(a))
print(min(b))

ica_g = ICAdecorr([a],[b],0.05, 10)
plt.plot(ica_g[0])
plt.title('ICA Correction of Motion Artefacts')
plt.xlabel('Frames')
plt.ylabel('Normalised Intensity')
# plt.show()
# plot_all(p)
# print(c)

**Evaluate FOOPSI Parameters**

In [ ]:
#Tuning for FOOPSI 

tuning_thresh_foopsi = 0.03

evaluation_neuron_foopsi = 0
spikes_signal_tuning = FOOPSI_all(normalize(detrended))

plt.figure(1)
plt.plot(detrended[evaluation_neuron_foopsi])
plt.title('detrended signal for Specified Neuron')
plt.ylabel('Calcium Signal Detrended Intensity')
plt.xlabel('Frame Number')
plt.show()

plt.figure(2)
plt.plot(spikes_signal_dR[evaluation_neuron_foopsi])
plt.title('FOOPSI Results for Specified Neuron')
plt.ylabel('Estimated Neural Activity')
plt.xlabel('Frame Number')
plt.show()

**Evaluate use of ICA or DeltaR/R to separate signal from noise**

In [ ]:
#ICA artifact removal comparison cell
ICA_test_neuron = 9
R = intensities_red
G = intensities

a = decorrelateNeuronsICA(R, G, 0.2)

plt.plot(G[ICA_test_neuron], c = 'g')
plt.plot(R[ICA_test_neuron], c = 'r')
plt.show()

plt.plot(a[ICA_test_neuron], c = 'g')
plt.title('ICA')
plt.show()

plt.plot(deltaR[ICA_test_neuron], c = 'r')
plt.title('DR/R')
plt.show()

**Find Raster Plot from Non-DR/R data**    
Recalculates the spikes from the raw data and plots the results      
Evaluates best input to use with FOOPSI Algorithm

In [ ]:
#raster with non-dR/R - Compare Rasters from different input data

Raw_filt, _ = np.asanyarray(filt_nematocytes(intensities, 5, positions))

raw_smooth = smoother(Raw_filt, 5)

df_f_signal = np.asanyarray(df_f(raw_smooth))


plt.figure(1)
for i in range(len(raw_smooth)):
    plt.plot(raw_smooth[i])
plt.title('Intensity Plot Raw smooth')
plt.xlabel('Frame')
plt.ylabel('Intensity')
plt.show()

#Create Raster Plot using CAIMAN's FOOPSI function
#foopsi (used on non-df/f data)
spikes_signal = np.zeros((len(Raw_filt),len(Raw_filt[1])))
for i in range(len(Raw_filt)):
    ca_foopsi,cb,b1,g,c1,spikes_foopsi,lam = deconv.constrained_foopsi(Raw_filt[i],p=2)
    spikes_signal[i] = spikes_foopsi
    
#foopsi (used on df/f data)
spikes_signal_df = np.zeros((len(df_f_signal),len(df_f_signal[1])))
for i in range(len(df_f_signal)):
    ca_foopsi,cb,b1,g,c1,spikes_foopsi,lam = deconv.constrained_foopsi(df_f_signal[i],p=2)
    spikes_signal_df[i] = spikes_foopsi

#threshold to find spikes
spike_thresh = 0.3

raster_array = np.zeros((len(Raw_filt),len(Raw_filt[1])))
for i in range(len(spikes_signal)):
    for j in range(len(spikes_signal[i])):
        if spikes_signal[i][j] >= spike_thresh*max(spikes_signal[i]):
            raster_array[i][j] = j

raster_array_df = np.zeros((len(df_f_signal),len(df_f_signal[1])))
for i in range(len(spikes_signal_df)):
    for j in range(len(spikes_signal_df[i])):
        if spikes_signal_df[i][j] >= spike_thresh*max(spikes_signal_df[i]):
            raster_array_df[i][j] = j

#Plot raster for base data
#zero values from way array was made array give large syncronous train at start of the signal - should fix (.append method?)
plt.figure(3)
plt.eventplot(raster_array,linelengths = 0.5)
plt.title('Raster Plot of All Neurons (raw intensities)')
plt.xlabel('Frame Number')
plt.ylabel('Neuron')
plt.show()

plt.figure(4)
plt.eventplot(raster_array_df,linelengths = 0.5)
plt.title('Raster Plot of All Neurons (df/f intensities)')
plt.xlabel('Frame Number')
plt.ylabel('Neuron')
plt.show()

**Alternative Clustering Visualisation**

In [ ]:
#Clustering Data - evaluate clustering with a cluster map (provides some extra visualisation if you want it but I prefer other methods)
#Input Number of Clusters Based on Results from Dendogram

eucdist = cdist(CaSignal_norm,CaSignal_norm)
cluster = AgglomerativeClustering(n_clusters=number_of_clusters, affinity='euclidean')
cluster.fit_predict(CaSignal_norm)
plt.figure(figsize=(10, 7))  
plt.scatter(eucdist[:,1], eucdist[:,0],c=cluster.labels_) 

**Evaluate Raster Reshaping**

In [ ]:
# #Evaluate Raster Reshaping step for saving as csv

# raster_reshape_test_neuron = 5

# plt.plot(raster_array_pred[raster_reshape_test_neuron], color = 'r')
# plt.eventplot(raster_array_dR[raster_reshape_test_neuron])

**Evaluate Neuron Tracking**

In [ ]:
n = 22
_,_,_,display = SingleCellIntensity(neuron = n, video = vid, positions = posit_corrected, dimentionROI = dim, Circle_radius = 4, distance_threshold = 4, display_on = True)

**Save Tracked Neurons**

In [ ]:
# for i in range(len(display)):
#     plt.imshow(display[i])
#     title = 'neuron_' + str(i) + '.tif'
#     plt.savefig(str(title), format = 'tif')

Experimental Cells
===============

In [ ]:
#raster using peak detection function - needs tuning...

neuron_for_peaks = 50
distance_between_peaks = 10
prominance_of_peaks = 0.05

a,b = sp.signal.find_peaks(detrended[neuron_for_peaks], distance = distance_between_peaks, prominence = prominance_of_peaks)
plt.figure(1)
plt.plot(detrended[neuron_for_peaks])
plt.plot(a, detrended[neuron_for_peaks][a], 'x')
plt.show()

raster2 = np.zeros((len(detrended), len(detrended[1])))
for i in range(len(detrended)):
    peaks,_ = sp.signal.find_peaks(detrended[i], distance = distance_between_peaks, prominence = prominance_of_peaks)
    for j in range(len(detrended[i])):
        for k in range(len(peaks)):
            raster2[i][peaks[k]] = 1

plt.figure(2)
plt.plot(raster2[neuron_for_peaks])
plt.show()

In [ ]:
#frequency filtering
#sampling rate = 3Hz
#hmm doesnt seem to work well at all

neurons = 10

frqs = np.fft.fft(detrended[neuron])
xax = np.arange(0,6000, 6000/(len(detrended[0])))
plt.figure(1)
plt.title('Fourier Spectrum')
plt.plot(abs(frqs.real))


#freqs adjusted
filter_block = np.ones(len(detrended[0]))
low_pass_cut = 30
filter_block[low_pass_cut:len(filter_block)-low_pass_cut] = 0
#filter_block[len(filter_block) - 80: len(filter_block) - 55] = 1
plt.plot(filter_block)
plt.show()
new_freqs = np.fft.ifft(frqs*filter_block)

plt.figure(2)
plt.plot(new_freqs)
plt.show()

In [ ]:
# #independant component analysis to separate noise - single neuron

# ica_test_neuron = 9
# window = 5

# data_green = [intensities[ica_test_neuron]]
# filtered_data_g = smoother(data_green,window)
# filtered_data_g = np.transpose(detrend(np.transpose(filtered_data_g), 17))
# plt.figure(1)
# plt.plot(filtered_data_g[0],c='g')
# # plt.plot(data_green[0],'r')

# data_red = [intensities_red[ica_test_neuron]]
# filtered_data_r = smoother(data_red,window)
# filtered_data_r = np.transpose(detrend(np.transpose(filtered_data_r), 17))
# plt.figure(2)
# plt.plot(filtered_data_r[0],c='r')
# # plt.plot(data_red[0],'r')
# plt.show()

# X = np.c_[filtered_data_g[0],filtered_data_r[0]]
# print(X.shape)
# X = X.reshape(len(filtered_data_g[0]),2)
# print(X.shape)
# # scaler= StandardScaler(copy=True, with_mean=True, with_std=True)
# # X = scaler.fit_transform(X)
# ica = FastICA(n_components = 2, random_state = None)
# A = ica.fit_transform(X)

# plt.figure(3)
# plt.plot(X[:,0], c='g')
# plt.plot(X[:,1], c='r')
# plt.plot(data_green, c='b')
# plt.plot(data_red, c='y')
# plt.figure(4)
# plt.plot(A[:,0])
# plt.plot(A[:,1])
# plt.show()

In [ ]:
# #Correlation Between Neural Activity and Behaviour
# #Method finds change in neural activity between CB and RP

# #select neuron to observe
# neuron = 44

# #select lenght of sub sequences (will depend on number of frames behaviour takes up)
# seqlen = 100

# df_on = smooth_intensities[neuron][301:565] + smooth_intensities[neuron][991:1076]
# df_ons = []
# for i in range(int(len(df_on)/seqlen)):
#     df_ons.append(df_on[i*seqlen:(i*seqlen)+seqlen])

# df_off = smooth_intensities[neuron][0:301] + smooth_intensities[neuron][565:991] + smooth_intensities[neuron][1076: len(smooth_intensities[0])]
# df_offs = []
# for i in range(int(len(df_off)/seqlen)):
#     df_offs.append(df_off[i*seqlen:(i*seqlen)+seqlen])
    
# #find average of on signal
# avg_on = []
# for i in range(len(df_ons[0])):
#     values = []
#     for j in range(len(df_ons)):
#         values.append(df_ons[j][i]) 
#     avg_on.append(np.mean(values))

# #find average of off signal
# avg_off = []
# for i in range(len(df_offs[0])):
#     values = []
#     for j in range(len(df_offs)):
#         values.append(df_offs[j][i]) 
#     avg_off.append(np.mean(values))

# #correlation
# corr_on_off = np.corrcoef(avg_off,avg_on)[0,1]
# print('Pearson Correlation Coeff: ', corr_on_off)

# #if correlation is very low, neurons change activity patterns during behaviour!!
# if abs(corr_on_off) < 0.5:
#     print('Neuron Changes Activity During Behaviour')
# else:
#     print('Neuron Does Not Change Activity During Behaviour')
    
# plt.plot(avg_on)
# plt.plot(avg_off)
# plt.show()